In [6]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import csv

#### Read Data In

In [25]:
import os
import re

path = "Android/Incoming/"
filelist = []
for f in os.listdir(path):
    if f[0] != '.' and 'LABELED' not in str(f):
        if re.search("[^_]*-[^_]*", f):
            day = re.search("[^_]*-[^_]*", f).group()
            filelist.append((f, day))
        
print filelist

[('Chart Andrew_03-19-2017_Evening_acc.csv from SensorKineticsPro', '03-19-2017'), ('Chart Andrew_3-20-2017_Morning_mag.csv from SensorKineticsPro', '3-20-2017'), ('Andrew_03-19-2017_Morning_gyr.csv', '03-19-2017'), ('Chart Andrew_03-19-2017_Evening_gyr.csv from SensorKineticsPro', '03-19-2017'), ('Chart Andrew_3-20-2017_Morning_gyr.csv from SensorKineticsPro', '3-20-2017'), ('Chart Andrew_03-19-2017_Evening_mag.csv from SensorKineticsPro', '03-19-2017'), ('Andrew_03-19-2017_Morning_mag.csv', '03-19-2017'), ('Chart Andrew_3-20-2017_Morning_acc.csv from SensorKineticsPro', '3-20-2017'), ('Andrew_03-18-2017_Evening_gyr.csv', '03-18-2017'), ('Andrew_03-18-2017_Evening_acc.csv', '03-18-2017'), ('Andrew_03-18-2017_Evening_mag.csv', '03-18-2017')]


#### Necessary Input

In [26]:
events_list = [("10:10:01", "W"), ("10:19:00", "T_B"), ("10:27:00", "T_E"), ("10:28:00", "P_W"), 
         ("10:31:00", "T_B"), ("10:32:00", "T_S"), ("10:35:00", "T_S"), ("10:38:00", "T_E"), 
         ("10:39:00", "W")]

In [27]:
days_and_starts = {'2017-03-19':["10:10:01", "18:12:00"], '2017-03-20': ['08:26:00', '']}

#### Functions for Classifying

In [28]:
def add_acceleration_magnitude(df):
    "Adds Acceleration magnitude to a dataframe. First 3 columns must be acceleration vectors. Must be 3D."
    from math import sqrt

    accel_mag = []
    
    for row in range(len(df)):
        accel_mag.append(sqrt(df.iloc[row, 0]**2 + df.iloc[row, 1]**2 + df.iloc[row, 2]**2))

    df['accel_mag'] = accel_mag
    return 'finished!'

In [29]:
def rel_time_to_real_time(df, day, start_time):
    from pandas.tseries.offsets import DateOffset
    df.index = [pd.Timestamp(day + " " + start_time) + DateOffset(seconds=x) for x in df.index]
    return "Finished!"

In [30]:
def add_classifications(df, day, events):
    '''Pass events as a list of tuples (datetime, eventcode) to classify different events '''
    from pandas.tseries.offsets import DateOffset
    
    for event in events:
        df.loc[day + " " + event[0], 'classification'] = event[1]
        if event[1] == "T_S":
            df.loc[str((pd.Timestamp(day + " " + event[0])+ pd.DateOffset(seconds=40))), 'classification'] = "T_B"
    df = df.fillna(method='ffill', inplace = True)
    
    return "Finished!"

In [34]:
def label_raw_android(f, day, events, start_time, accel_mag = False):
    df = pd.read_csv((path + f), index_col = 0)
    if accel_mag == True:
        add_acceleration_magnitude(df)
    rel_time_to_real_time(df, day, start_time)
    df['classification'] = None
    add_classifications(df, day, events)
    df.to_csv("Android/Classified/LABELED_" + f)
    print "Finished!"

#### Classifying Data

In [35]:
#Use this function to Classify an android file. Must be done manually now... will automate
#later if we decide to use a lot of android data

label_raw_android("Andrew_03-19-2017_Morning_mag.csv", "2017-03-19", events_list, "10:10:01")

Finished!
